In [ ]:
import numpy as np
from ml_lib.atlas_ml import *
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time

In [ ]:
class MNIST_CNN:
    def __init__(self, Y_size, lossfn, n_channels=1):
        self.L1 = conv_layer([5,5], n_channels, leaky_relu, n_filters = 10,padding = [2,2])
        self.L2 = conv_layer([3,3], 10, leaky_relu, n_filters = 15)
        self.L3 = max_pool_layer([2,2],15, stride=[2,2])
        self.L4 = conv_layer([3,3], 15, leaky_relu)        
        self.L5 = layer(121,50, leaky_relu)
        self.L6 = layer(50, Y_size, softmax)
        self.lossfn = lossfn()
        
    def f_pass(self, X):
        A = self.L1.forward(X)
        A = self.L2.forward(A)
        A = self.L3.forward(A)
        A = self.L4.forward(A)
        A.resize(A.shape[0], A.shape[2]*A.shape[3])
        A = self.L5.forward(A)
        A = self.L6.forward(A)
        self.H = A
        return self.H
    
    def back_prop(self,X,Y, batch_size,reg_lambda=0):
        m = batch_size
        self.loss = self.lossfn.get_loss(self.H,Y)
        dZ = self.lossfn.diff(self.H,Y)
        dA = self.L6.out_grad(dZ, self.L5.A, m)
        dA = self.L5.grad(dA,self.L4.A, m)
        dA = np.expand_dims(dA,axis=-1)
        dA = self.L4.grad(dA)
        dA = self.L3.grad(dA)
        dA = self.L2.grad(dA)
        dX = self.L1.grad(dA)
    
    def optim(self, lr, beta=0):
        self.L1.step(lr,beta)
        self.L2.step(lr,beta)
        self.L3.step(lr,beta)
        self.L4.step(lr,beta)
        self.L5.step(lr,beta)
        self.L6.step(lr,beta)

In [ ]:
trainX_path = '../data/fashion_mnist/train-images-idx3-ubyte'
trainY_path = '../data/fashion_mnist/train-labels-idx1-ubyte'
testX_path  = '../data/fashion_mnist/t10k-images-idx3-ubyte'
testY_path  = '../data/fashion_mnist/t10k-labels-idx1-ubyte'

X,Y,X_test,Y_test = load_mnist_data(trainX_path,trainY_path,testX_path,testY_path)

In [ ]:
n_out = np.shape(Y)[1]
mnist_cnn = MNIST_CNN(n_out,CE_loss)

In [ ]:
X.shape

In [ ]:
batch_size = 16

lr = 0.005

n_epochs = 10

lr_decay = 0.9

data_size = X.shape[0]

beta = 0

In [ ]:
train(mnist_cnn, X, Y, X_test, Y_test, model_accuracy, n_epochs, \
    batch_size, lr, lr_decay, beta)

In [ ]:
# measure time taken
start = time. time()
mnist_cnn.f_pass(X[0:4])
endf = time. time()
mnist_cnn.back_prop(X[0:4],Y[0:4],4)
endb = time. time()
mnist_cnn.optim(lr,beta)
endo = time. time()
mnist_cnn.loss

In [ ]:
print(f"f-pass:{endf-start:.4f} | b-prop:{endb-endf:.4f} | optim: {endo-endb:.4f} | total: {endo-start:.4f}")

In [ ]:
import pickle

# Does not work currently

def save_model(model, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(model, output, pickle.HIGHEST_PROTOCOL)

def load_model(filename):
    model = None
    with open(filename, 'r') as f:  # Overwrites any existing file.
        model = pickle.load(f)   
    return model